In [6]:
import cv2

# Load the main video (VideoCapture 0 usually refers to the webcam on a computer)
cap = cv2.VideoCapture(0)

# Load the second video
cap2 = cv2.VideoCapture('Test.mp4')

pressed_k = False
while True:
    # Read the main video
    ret, frame = cap.read()
    if not ret:
        break

    # Display the main video
    cv2.imshow('Main Video', frame)

    # Get the key press
    key = cv2.waitKey(1) & 0xFF
    if(key == ord('k')):
         cap2.release()
         cap2 = cv2.VideoCapture('Test.mp4')
         
    ret2, frame2 = cap2.read()
    if not ret2:
            # Reset the capture if the video has ended
            cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue
    cv2.imshow('Second Video', frame2)

    # Quit if 'q' is pressed
    if key == ord('q'):
        break

# Release the video captures
cap.release()
cap2.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


In [ ]:
import tkinter as tk
from tkinter import ttk

def open_window():
    # Create a new Tkinter window
    window = tk.Tk()
    window.title("Tkinter Window")

    # Create the notebook (tabs container)
    notebook = ttk.Notebook(window)

    # Create frames for each tab
    frame1 = ttk.Frame(notebook)
    frame2 = ttk.Frame(notebook)
    frame3 = ttk.Frame(notebook)

    # Add frames to notebook
    notebook.add(frame1, text="Tab 1")
    notebook.add(frame2, text="Tab 2")
    notebook.add(frame3, text="Tab 3")

    # Add content to tabs
    label1 = tk.Label(frame1, text="This is Tab 1")
    label1.pack()
    label2 = tk.Label(frame2, text="This is Tab 2")
    label2.pack()
    label3 = tk.Label(frame3, text="This is Tab 3")
    label3.pack()

    # Pack the notebook
    notebook.pack(expand=True, fill='both')

    # Run the window's main loop
    window.mainloop()

# Call the function to open the window
open_window()


In [4]:
import json
import csv
import numpy as np
import pandas as pd
from dtw import dtw
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [5]:

#Starting point
with open('Gestures.csv', mode='w', newline='') as file:
      writer = csv.writer(file)
      starting_row = ['label' , 'trajectory']
      writer.writerow(starting_row)


In [31]:

def Training_classify_traj(ex_name):

    lower_bound = (31, 60, 87)
    upper_bound = (61, 255, 219)

    lower_bound = (169 , 148 , 92)
    upper_bound = (179 , 255 ,255)
    object_track = False
    cap = cv2.VideoCapture(0)  # Start video capture
    trajectory_points = []
    count = 0
    while True:
        count += 1
        if(count == 80):
            break
        ret, frame = cap.read()  # Read frame from the camera
        frame = cv2.flip(frame, 1)
        if ret:  # If frame read successful
            # Convert image color to HSV (easier for color detection)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Create a binary image where white pixels are those within the color range
            mask = cv2.inRange(hsv, lower_bound, upper_bound)

            # Find the contours of the binary image
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Filter contours based on area
            filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 100]

            if filtered_contours:
                largest_contour = max(filtered_contours, key=cv2.contourArea)
                object_track = True
                # Get the coordinates of the center of the largest contour
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    trajectory_points.append((cX,cY))
                else:
                    cX, cY = 0, 0

                # Draw the center of the shape on the image
                cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
                cv2.putText(frame, "center", (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            else:
                if(object_track == True):
                    print(object_track)
                object_track = False
                
            # Display frame
            cv2.imshow("Frame", frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit if 'q' is pressed
                break

        else:
            break


    cap.release()  # Release video capture
    cv2.destroyAllWindows()  # Close all OpenCV windows

    with open('Gestures.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        traj = json.dumps(trajectory_points)
        row = [ex_name , traj]
        #writer.writerow(row)

Training_classify_traj('Start_2')


In [84]:
#Reading Phase
traj_info = pd.read_csv('Gestures.csv')
map_traj = {}
for index, row in traj_info.iterrows():
      label = str(row['label'])
      trajectories = eval(row['trajectory'])
      trajectories = np.array(trajectories)
      map_traj[label] = trajectories
print(map_traj)

{'Left_1': array([[ 82, 207],
       [ 80, 207],
       [ 80, 207],
       [ 80, 207],
       [ 80, 207],
       [ 80, 209],
       [ 80, 209],
       [ 80, 209],
       [ 80, 209],
       [ 80, 210],
       [ 80, 210],
       [ 80, 210],
       [ 80, 210],
       [115, 216],
       [115, 216],
       [115, 216],
       [115, 216],
       [172, 219],
       [172, 219],
       [172, 219],
       [172, 219],
       [224, 246],
       [224, 246],
       [224, 246],
       [224, 246],
       [274, 242],
       [274, 242],
       [274, 242],
       [274, 242],
       [313, 237],
       [313, 237],
       [313, 237],
       [313, 237],
       [353, 225],
       [353, 225],
       [353, 225],
       [353, 225],
       [396, 219],
       [396, 219],
       [396, 219],
       [396, 219],
       [449, 208],
       [449, 208],
       [449, 208],
       [449, 208],
       [474, 193],
       [474, 193],
       [474, 193],
       [474, 193],
       [486, 169],
       [486, 169],
       [486, 169],
 

In [85]:
from dollarpy import Recognizer, Template, Point
trajectories_to_classify = [[6,4],[1,2]]
def classify(Temp , tc):
    min_distance = float('inf')
    class_label = None
    for traj in Temp:
      distance, path = fastdtw(tc, Temp[traj] , dist=euclidean)
      if(distance < min_distance):
         min_distance = distance
         class_label = traj
    return min_distance , class_label

def dollar_Templates():
  Templates_dollar_py = []
  for k in map_traj:
    curr_traj = map_traj[k]
    L = []
    for c in curr_traj:
        p = Point(c[0] , c[1])
        L.append(p)
    print(k)
    print(L)
    template_dpy = Template(k , L)
    Templates_dollar_py.append(template_dpy)
  return Templates_dollar_py
temps = dollar_Templates()
recognizer = Recognizer(temps)
   
#classify(Template , trajectories_to_classify)


Left_1
[(82, 207), stroke None, (80, 207), stroke None, (80, 207), stroke None, (80, 207), stroke None, (80, 207), stroke None, (80, 209), stroke None, (80, 209), stroke None, (80, 209), stroke None, (80, 209), stroke None, (80, 210), stroke None, (80, 210), stroke None, (80, 210), stroke None, (80, 210), stroke None, (115, 216), stroke None, (115, 216), stroke None, (115, 216), stroke None, (115, 216), stroke None, (172, 219), stroke None, (172, 219), stroke None, (172, 219), stroke None, (172, 219), stroke None, (224, 246), stroke None, (224, 246), stroke None, (224, 246), stroke None, (224, 246), stroke None, (274, 242), stroke None, (274, 242), stroke None, (274, 242), stroke None, (274, 242), stroke None, (313, 237), stroke None, (313, 237), stroke None, (313, 237), stroke None, (313, 237), stroke None, (353, 225), stroke None, (353, 225), stroke None, (353, 225), stroke None, (353, 225), stroke None, (396, 219), stroke None, (396, 219), stroke None, (396, 219), stroke None, (396,

In [88]:
def Test_Trajectory(Temp):
    lower_bound = np.array([155, 50, 50])
    upper_bound = np.array([175, 255, 255])

  
    object_track = False
    cap = cv2.VideoCapture(0)  # Start video capture
    trajectory_points = []
    trajectory_points_dp = []
    count = 0
    while True:
        count += 1
        if(count % 80 == 0):
            if trajectory_points_dp:
                result = recognizer.recognize(trajectory_points_dp)
                trajectory_points_dp = []
                print(result)
            
            
        ret, frame = cap.read()  # Read frame from the camera
        frame = cv2.flip(frame, 1)
        if ret:  # If frame read successful
            # Convert image color to HSV (easier for color detection)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Create a binary image where white pixels are those within the color range
            mask = cv2.inRange(hsv, lower_bound, upper_bound)

            # Find the contours of the binary image
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Filter contours based on area
            filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]

            if filtered_contours:
                # Get the largest contour
                largest_contour = max(filtered_contours, key=cv2.contourArea)
                object_track = True
                # Get the coordinates of the center of the largest contour
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    trajectory_points.append((cX,cY))
                    trajectory_points_dp.append(Point(cX , cY))
                else:
                     cX, cY = 0, 0

                # Draw the center of the shape on the image
                cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
                cv2.putText(frame, "center", (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
                
            # Display frame
            cv2.imshow("Frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit if 'q' is pressed
                break

        else:
            break
    cap.release()
    cv2.destroyAllWindows()
Test_Trajectory(map_traj)

('Start_1', 0.3019019872291987)
('Left_1', 0.33540158783473417)
('Start_1', 0.29674873896622367)
('Left_1', 0.40995351695049687)
('Start_1', 0.022637948337813407)
('Right_1', 0.2333152473912674)
(None, 0)
('Right_1', 0.11445276456974285)
('Left_1', 0.32513025814803587)
('Start_1', 0.27974140510199474)
('Left_1', 0.18939131240163365)
('Left_1', 0.08407830115773773)
('Right_1', 0.048349690898084274)
('Right_1', 0.21832255347399232)
(None, 0)
(None, 0)
(None, 0)
(None, 0)
('Start_1', 0.18084242206564738)
('Left_1', 0.539130930004864)


In [57]:
def Test_Trajectory(Temp):
    lower_bound = np.array([155, 50, 50])
    upper_bound = np.array([175, 255, 255])
    object_track = False
    cap = cv2.VideoCapture(0)  # Start video capture
    trajectory_points = []
    trajectory_points_dp = []
    count = 0
    while True:
        count += 1
        if(count % 50 == 0):
            if trajectory_points_dp:
               result = recognizer.recognize(trajectory_points_dp) 
               print(result)
            trajectory_points = []
            trajectory_points_dp = []
            
        ret, frame = cap.read()  # Read frame from the camera
        frame = cv2.flip(frame, 1)
        if ret:  # If frame read successful
            # Convert image color to HSV (easier for color detection)
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Create a binary image where white pixels are those within the color range
            mask = cv2.inRange(hsv, lower_bound, upper_bound)

            # Find the contours of the binary image
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Filter contours based on area
            filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]

            if filtered_contours:
                # Get the largest contour
                largest_contour = max(filtered_contours, key=cv2.contourArea)
                object_track = True
                # Get the coordinates of the center of the largest contour
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    trajectory_points.append((cX,cY))
                    trajectory_points_dp.append(Point(cX , cY, 1))
                else:
                     cX, cY = 0, 0

                # Draw the center of the shape on the image
                cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)
                cv2.putText(frame, "center", (cX - 20, cY - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
               
                
            # Display frame
            cv2.imshow("Frame", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit if 'q' is pressed
                break

        else:
            break
    cap.release()
    cv2.destroyAllWindows()
Test_Trajectory(map_traj)

('left_2', 0.3696272812350322)
('left_1', 0.5807411137686531)
('right_2', 0.32295478019449597)
('right_2', 0.1263593447222784)
('right_2', 0.26707090979021253)
('left_1', 0.5568588597747904)
('left_1', 0.8223358447106035)
('left_2', 0.005003643989261342)
(None, 0)
(None, 0)
('Start_2', 0.029042380597822604)
(None, 0)
(None, 0)
('right_1', 0.021401661430474284)
(None, 0)
('left_2', 0.4914467459705194)
(None, 0)
('Start_2', 0.000610463490223534)
('left_1', 0.21439816253854238)
('Start_2', 0.10107444819098177)


In [6]:
import cv2
import numpy as np

# Global variables
rectangle = False   # True if the left mouse button is DOWN
ix, iy = -1, -1     # Initial mouse click position
fx, fy = -1, -1     # Final mouse release position

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global rectangle, ix, iy, fx, fy

    if event == cv2.EVENT_LBUTTONDOWN:   # When the left mouse button is clicked DOWN
        rectangle = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:   # When the mouse is MOVING
        if rectangle == True:    # If the left button is DOWN (dragging the mouse)
            fx, fy = x, y
            cv2.rectangle(img, (ix, iy), (fx, fy), (0, 255, 0), 1)

    elif event == cv2.EVENT_LBUTTONUP:   # When the left mouse button is released UP
        rectangle = False
        fx, fy = x, y
        cv2.rectangle(img, (ix, iy), (fx, fy), (0, 255, 0), 1)

        # Here we can get the ROI and compute the average color
        roi = img[iy:fy, ix:fx]
        avg_color_per_row = np.average(roi, axis=0)
        avg_color = np.average(avg_color_per_row, axis=0)
        print(f'Average color in BGR format: {avg_color}')

        # Convert ROI from BGR to HSV
        roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        avg_color_per_row_hsv = np.average(roi_hsv, axis=0)
        avg_color_hsv = np.average(avg_color_per_row_hsv, axis=0)
        print(f'Average color in HSV format: {avg_color_hsv}')


# Create a black image and a window
img = np.zeros((512, 512, 3), np.uint8)
cv2.namedWindow('image')

# Bind the draw_rectangle function to mouse cliks and movement
cv2.setMouseCallback('image', draw_rectangle)

cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()  # Capture frame from the camera

    # Show the image in an OpenCV window
    cv2.imshow('image', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit if 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()


Average color in BGR format: [ 85.57081769 153.98664998 168.39945766]
Average color in HSV format: [ 23.25917814 128.48779725 173.58562787]


In [48]:
import cv2
import numpy as np
import imutils
from collections import deque
import time 
# Define the lower and upper boundaries of the "green" color in the HSV color space


rgb_tennis_ball = np.uint8([[[216, 214, 159]]])

# Convert the RGB color to HSV color
hsv_tennis_ball = cv2.cvtColor(rgb_tennis_ball, cv2.COLOR_RGB2HSV)

# Now we have the HSV value of the tennis ball
# We need to define a range around this value for color detection
# We can define a reasonable range as 10 for H and 40 for S and V
h, s, v = int(hsv_tennis_ball[0][0][0]), int(hsv_tennis_ball[0][0][1]), int(hsv_tennis_ball[0][0][2])
lower_bound = (max(0, h - 10), max(0, s - 40), max(0, v - 40))
upper_bound = (min(179, h + 10), min(255, s + 40), min(255, v + 40))
'''
greenLower = (18, 98, 143)
greenUpper = (28, 158, 203)
'''
greenLower = lower_bound
greenUpper = upper_bound
# Initialize the list of tracked points
pts = deque(maxlen=64)

# Start the video capture
cap = cv2.VideoCapture(0)

# Allow the camera to warm up
time.sleep(2.0)

# Keep looping
while True:
    # Grab the current frame
    _, frame = cap.read()
    
    # Resize the frame, blur it, and convert it to the HSV color space
    frame = imutils.resize(frame, width=600)
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Construct a mask for the color "green", then perform a series of dilations and erosions to remove any small blobs left in the mask
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

    # Find contours in the mask and initialize the current (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None

    # Only proceed if at least one contour was found
    if len(cnts) > 0:
        # Find the largest contour in the mask, then use it to compute the minimum enclosing circle and centroid
        c = max(cnts, key=cv2.contourArea)
        print(cv2.contourArea(c))
        if cv2.contourArea(c) > 200:  # Set your minimum contour area here
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            # Only proceed if the radius meets a minimum size
            if radius > 10:
                # Draw the circle and centroid on the frame, then update the list of tracked points

                cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

    # Update the points queue
    pts.appendleft(center)

    # Loop over the set of tracked points
    for i in range(1, len(pts)):
        # If either of the tracked points are None, ignore them
        if pts[i - 1] is None or pts[i] is None:
            continue

        # Otherwise, compute the thickness of the line and draw the connecting lines
        thickness = int(np.sqrt(64 / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

    # Show the frame
    cv2.imshow("Frame", frame)

    # Check for key press to exit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


27.0
27.0
27.0
27.0
30.0
30.0
30.0
30.0
150.0
150.0
150.0
150.0
821.0
821.0
821.0
821.0
584.5
584.5
584.5
584.5
1374.5
1374.5
1374.5
1374.5
1875.0
1875.0
1875.0
1875.0
2004.5
2004.5
2004.5
2004.5
3136.0
3136.0
3136.0
3136.0
3979.5
3979.5
3979.5
3979.5
5218.0
5218.0
5218.0
5218.0
9856.5
9856.5
9856.5
9856.5
9763.5
9763.5
9763.5
9763.5
9799.5
9799.5
9799.5
9799.5
10374.5
10374.5
10374.5
10374.5
9695.0
9695.0
9695.0
9695.0
10068.5
10068.5
10068.5
10068.5
9965.5
9965.5
9965.5
9965.5
9664.5
9664.5
9664.5
9664.5
8855.0
8855.0
8855.0
8855.0
7484.0
7484.0
7484.0
7484.0
5834.0
5834.0
5834.0
5834.0
4956.5
4956.5
4956.5
4956.5
2679.0
2679.0
2679.0
2679.0
2882.0
2882.0
2882.0
2882.0
4901.5
4901.5
4901.5
4901.5
6921.0
6921.0
6921.0
6921.0
8274.0
8274.0
8274.0
8274.0
8159.0
8159.0
8159.0
8159.0
7879.0
7879.0
7879.0
7879.0
4772.5
4772.5
4772.5
4772.5
7451.0
7451.0
7451.0
7451.0
7021.0
7021.0
7021.0
7021.0
6642.5
6642.5
6642.5
6642.5
4132.5
4132.5
4132.5
4132.5
7287.0
7287.0
7287.0
7287.0
7792.0
7792.

In [7]:
import numpy as np

# Define the HSV color range of the green ball
h_range = 5  # Adjust this value as needed
s_range = 30  # Adjust this value as needed
v_range = 30  # Adjust this value as needed

# Convert the average HSV color values to integers
avg_color_hsv = np.array([23,128,173], dtype=int)

# Calculate the lower and upper boundaries based on the average HSV color values
greenLower = np.array([max(0, avg_color_hsv[0] - h_range), max(0, avg_color_hsv[1] - s_range), max(0, avg_color_hsv[2] - v_range)])
greenUpper = np.array([min(180, avg_color_hsv[0] + h_range), min(255, avg_color_hsv[1] + s_range), min(255, avg_color_hsv[2] + v_range)])

# Print the updated HSV color range for the green ball
print(f"Updated greenLower: {greenLower}")
print(f"Updated greenUpper: {greenUpper}")


Updated greenLower: [ 18  98 143]
Updated greenUpper: [ 28 158 203]
